In this notebook I test the ability for users with role "TEAM_DEV_ADMIN" in snowflake to manipulate data in the RAW.TEAM_DEV schema. 

Specifically, they should be able to:
- create a new table
- insert new rows
- perform select operations
- delete rows 
- drop table

## environment setup

In [35]:
from utils_snowflake import load_config, create_snowflake_engine
from sqlalchemy import Table, Column, Integer, Float, String, MetaData, Sequence
from sqlalchemy import select, delete, text
from sqlalchemy.sql import insert

CONFIG = load_config()["snowflake"]
account = CONFIG["account"]
username = CONFIG["team_dev_admin"]["username"]
password = CONFIG["team_dev_admin"]["password"]
warehouse = CONFIG["team_dev_admin"]["warehouse"]
role = CONFIG["team_dev_admin"]["role"]
database = "RAW"
schema = "TEAM_DEV"

engine = create_snowflake_engine(
            database=database, 
            schema=schema,
            user=username,
            password=password,
            warehouse=warehouse,
            role=role)

## create a table

In [36]:
metadata = MetaData()
table_name = 'test_table'

# Define a sequence for the auto-incrementing ID
id_seq = Sequence('id_seq')

table = Table(table_name, metadata,
              Column('id', Integer, id_seq, server_default=id_seq.next_value(), primary_key=True),
              Column('a', Integer),
              Column('b', Float),
              Column('c', String)
              )

# Create the table in Snowflake
metadata.create_all(engine)

## insert rows

In [41]:
# Insert data
with engine.connect() as connection:
    ins_query = insert(table).values(a=2, b=3.5, c="hello")
    connection.execute(ins_query)

## select

In [42]:
query = text(f"SELECT * FROM {table_name}")
# Execute the query
with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)

(1, 2, 3.5, 'hello')
(2, 2, 3.5, 'hello')
(3, 2, 3.5, 'hello')
(4, 2, 3.5, 'hello')
(5, 2, 3.5, 'hello')


## delete rows

In [31]:
# Define the DELETE query as a textual SQL expression
delete_query = text(f"DELETE FROM {table_name} WHERE id = 2")

# Execute the DELETE query
with engine.connect() as connection:
    connection.execute(delete_query)

## drop table

In [33]:
# Drop the table
with engine.connect() as connection:
    connection.execute(f'DROP TABLE IF EXISTS {table_name}')